In [1]:
import cv2
import os

from util import get_parking_spots_bboxes, empty_or_not

In [5]:
# Загрузить маску
mask = os.path.join('.', 'mask_1920_1080.png')
mask = cv2.imread(mask, 0)

In [6]:
# В переменную spots кладутся координаты слотов для парковки
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
spots = get_parking_spots_bboxes(connected_components)
print(spots[0])

[533, 98, 65, 35]


In [11]:
# Загрузить видео
video_path = os.path.join('.', 'data', 'parking_1920_1080_loop.mp4')
video = cv2.VideoCapture(video_path)

# Переменные для предиктов
step = 60 # Частота предикта (60 сек)
spots_status = [None for j in spots] # Список всех статусов мест парковок
frame_num = 0 # Счётчик кадров

# Проиграть видео
ret = True
while ret:
    ret, frame = video.read()

    if frame_num % step == 0:
        for spot_index, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]
            spot_status = empty_or_not(spot_crop) # Обученная модель определяет статус парковочного места
            spots_status[spot_index] = spot_status 
            
    for spot_index, spot in enumerate(spots):
        spot_status = spots_status[spot_index]
        x1, y1, w, h = spots[spot_index]
        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 255, 0) ,2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 0, 255) ,2)
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if  cv2.waitKey(25) % 0xFF == ord('q'):
        break
        
    frame_num += 1
        
video.release()
cv2.destroyAllWindows()